In [1]:
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.datasets import Omniglot
from torchvision.transforms import transforms
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score

/home/nekozo/anaconda3/envs/pytorch/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
ds = Omniglot(root='.', download=True, transform=transform)

100.0%


Extracting ./omniglot-py/images_background.zip to ./omniglot-py
